In [1]:
import glob
import shutil
from tqdm.notebook import tqdm
import random
import pandas as pd
import numpy as np

import os
import json


In [2]:
"""Takes in bounding box data, contatining class
   and position and converts it to the correct format"""

def convert_format(data, width, height):
    bbox = [float(elem) for elem in data[1:]]
    x, y, w, h = bbox
    xmin = int((x - w / 2) * width)
    ymin = int((y - h / 2) * height)
    xmax = int((x + w / 2) * width)
    ymax = int((y + h / 2) * height)
    return class_index+" "+" ".join([str(xmin), str(ymin), str(xmax), str(ymax)])

In [3]:
image_list = glob.glob('RDD2022_all_countries/Norway/test/images/*.jpg')

In [4]:
from PIL import Image

subm_file = open("submissiwe.txt", "w")
for file in tqdm(image_list):

    basename = os.path.basename(file)
    annot_name = basename.split(".")[0]+".txt"
    annot_path = "yolov7/runs/detect/preds_yolov7normal/labels/"+annot_name
    
    with Image.open(file) as img:
        width, height = img.size
    
    content = []
    if os.path.exists(annot_path):
        with open(annot_path, "r") as f:
            data = [line.strip().split(" ") for line in f.readlines()]
            for elem in data:
                content.append(convert_format(elem, width, height))
    else:
        content.append("")
        
    subm_file.write(basename+","+" ".join(content)+"\n")
    
subm_file.close()
    

  0%|          | 0/2040 [00:00<?, ?it/s]

In [44]:
import cv2
import matplotlib.pyplot as plt

def plot_bounding_box(image_path, width, height):
    img = cv2.imread(image_path)
    dh, dw, _ = img.shape

    split_string = image_path.split('\\')
    

    
    label_path = '\\'.join((split_string[:2]))+'\\labels\\'+split_string[3].split('.')[0]+'.txt'
    fl = open(label_path, 'r')
    data = fl.readlines()
    fl.close()

    for dt in data:

        # Split string to float
        _, x, y, w, h = map(float, dt.split(' '))

        # Taken from https://github.com/pjreddie/darknet/blob/810d7f797bdb2f021dbe65d2524c2ff6b8ab5c8b/src/image.c#L283-L291
        # via https://stackoverflow.com/questions/44544471/how-to-get-the-coordinates-of-the-bounding-box-in-yolo-object-detection#comment102178409_44592380
        l = int((x - w / 2) * dw)
        r = int((x + w / 2) * dw)
        t = int((y - h / 2) * dh)
        b = int((y + h / 2) * dh)

        if l < 0:
            l = 0
        if r > dw - 1:
            r = dw - 1
        if t < 0:
            t = 0
        if b > dh - 1:
            b = dh - 1

        cv2.rectangle(img, (l, t), (r, b), (0, 0, 255), 1)

    plt.imshow(img)
    plt.show()



In [ ]:
plot_bounding_box('dataset\\train\\images\\000015.jpg')

In [ ]:
plot_bounding_box('dataset\\train\\images\\000028.jpg')

In [ ]:
plot_bounding_box('dataset\\train\\images\\002008.jpg')